In [1]:
"""A script to handle kaggle FGVC5 json files

   moduleauthor:: InCodeWeTrust

"""

import requests
import time
import json
import pandas as pd
# from PIL import Image
# from io import BytesIO

JSON_FILE_PATH = "/home/wyao/github/InCodeWeTrust/project_FGVC5/data/input/"
IMAGE_PATH = "/home/wyao/github/InCodeWeTrust/project_FGVC5/data/working/"
TRAIN_FILE_NAME = "train.json"
TEST_FILE_NAME = "test.json"
VALID_FILE_NAME = "validation.json"
MAX_RETRY = 3

train_image_file = JSON_FILE_PATH + TRAIN_FILE_NAME
valid_image_file = JSON_FILE_PATH + VALID_FILE_NAME
test_image_file = JSON_FILE_PATH + TEST_FILE_NAME
# print(image_url_file)

In [2]:
def parse_json(json_file, json_key):
    """This function extracts image urls from json file.
    
    :param json_file: the full path of json file
    :type name: str
    :param json_key: component to be extract from input json file. 
                Possible values are ['images','annotation'].
    :type json_key: str
    :returns: pandas data frame with image id and url.
    
    """
    with open(train_image_file, 'r') as f:
        image_dict = json.load(f)
    print(image_dict.keys())

    if (json_key not in ['images','annotations']):
        raise ValueError("Invalid input argument: json_key.")
    if (json_key == 'images'):
        image_url_df = pd.DataFrame(image_dict['images'])
        image_url_df.url = image_url_df.url.apply(lambda x: x[0])
        print(image_url_df.head())
        return image_url_df
    elif (json_key == 'annotations'):
        image_annotation_df = pd.DataFrame(image_dict['annotations'])
        print(image_annotation_df.head())
        return image_annotation_df


In [3]:
image_url_df = parse_json(train_image_file, 'images')
image_annotations_df = parse_json(train_image_file, 'annotations')
image_annotations_df.to_csv(JSON_FILE_PATH + "train_label.csv",
                            index=False)

dict_keys(['annotations', 'images'])
   image_id                                                url
0         1  https://img13.360buyimg.com/imgzone/jfs/t2857/...
1         2  http://www.tengdakeli.cn/350/timg01/uploaded/i...
2         3  https://img13.360buyimg.com/imgzone/jfs/t8899/...
3         4  http://img4.tbcdn.cn/tfscom/i1/2855447419/TB2S...
4         5  http://a.vpimg4.com/upload/merchandise/287883/...
dict_keys(['annotations', 'images'])
   image_id  label_id
0         1         5
1         2         5
2         3         5
3         4         5
4         5         5


In [4]:
def show_image_by_id(image_dataframe, image_id, save = False):
    """This function show image by id.
    
    :param image_dataframe: a data frame contains image id and image url.
    :type image_dataframe: pandas data frame
    :param image_id: a positive integer that represents image id
    :type image_id: integer
    :parame save: boolean argument indicating if the image will be saved
    :type save: boolean
    :returns: image object
    
    """
    image_index = image_id - 1

    url = image_dataframe.iloc[image_index]['url']
    if url.find('/'):
        original_image_name = url.rsplit('/', 1)[1]
        image_data = requests.get(url).content
        new_image_name = str(image_id) + '.jpg'
        new_image_path = IMAGE_PATH + new_image_name
        if (save):
            with open(new_image_path, 'wb') as handler:
                handler.write(image_data)
            print("Saved image: {} as {}.".format(original_image_name,
                                                  new_image_name))
        else:
            img = Image.open(BytesIO(image_data))
            img.show()
    else:
        print("Image is not found.")

show_image_by_id(image_url_df, 4, True)

Saved image: TB2SJwwdxeK.eBjSZFlXXaywXXa_!!2855447419.jpg as 4.jpg.


In [9]:
def save_image_batch(image_dataframe, dest, id_start_from=1):
    """This function show image in batch.
    
    :param image_dataframe: a data frame contains image id and image url.
    :type image_dataframe: pandas data frame
    :param dest: dest possible values: ["train","test","validation"]
    :type dest: str
    :param id_start_from: image id starts from, has to be positive integer.
    :type id_start_from: int
    :returns: 
    
    """
    num_image = image_dataframe.shape[0]
    if dest == "train":
        saving_path = IMAGE_PATH + "train/"
    elif dest == "test":
        saving_path = IMAGE_PATH + "test/"
    elif dest == "validation":
        saving_path = IMAGE_PATH + "validation/"
    else:
        raise ValueError("invalid destination arg.")

    for idx in range(id_start_from-1, num_image):
        image_id = idx + 1
        url = image_dataframe.iloc[idx]['url']

        original_image_name = url.rsplit('/', 1)[1]
        retry = 0
        image_retrieved = True
        while retry < MAX_RETRY:
            try:
                image_data = requests.get(url,timeout=10).content
                break
            except:
                retry += 1
                print("Connection refused by the server...wait for 5 seconds")
                time.sleep(5)
                if retry == MAX_RETRY:
                    image_retrieved = False
                    print("Failed to get {}.jpg".format(image_id))
                continue

        if image_retrieved == True:
            new_image_name = str(image_id) + '.jpg'
            new_image_path = saving_path + new_image_name

            with open(new_image_path, 'wb') as handler:
                handler.write(image_data)
#             print("Saved image: {} as {}.".format(original_image_name,
#                                                   new_image_name))
    print("DONE!")


In [ ]:
# save training image
save_image_batch(image_url_df,"train",1)

Saved image: 57171f60N523c940e.jpg as 1.jpg.
Saved image: T1pFCtFbVcXXXXXXXX_%21%210-item_pic.jpg as 2.jpg.
Saved image: 59b25f42N52ee2b0c.jpg as 3.jpg.
Saved image: TB2SJwwdxeK.eBjSZFlXXaywXXa_!!2855447419.jpg as 4.jpg.
Saved image: QQI-390870207-1.jpg as 5.jpg.
Saved image: TB1HoBuHFXXXXXOXFXXXXXXXXXX_!!0-item_pic.jpg as 6.jpg.
Saved image: 6016537.jpg as 7.jpg.
Saved image: 5672f5e9N7dfa5a0e.jpg%21q70.jpg as 8.jpg.
Saved image: 683fc19f3fbdeaeb6af8bdee95ca5fd0.jpg as 9.jpg.
Saved image: 75ecb96db31337096ac2a9df85f00663.jpg as 10.jpg.
Saved image: 58429ddeN528837b7.jpg as 11.jpg.
Saved image: ChEi21ZD8omAGcyNAAN5x93OdSQ41401_360x360.jpg as 12.jpg.
Saved image: 55b8393aNf194fd7e.jpg%21q70.jpg as 13.jpg.
Saved image: TB2Fl3nbv6H8KJjSspmXXb2WXXa_!!2172811710.jpg as 14.jpg.
Saved image: 59340a23N7191bc5b.jpg as 15.jpg.
Saved image: 575d3181Na7521aa3.jpg as 16.jpg.
Saved image: 543c90eaNeb4ac6c1.jpg as 17.jpg.
Saved image: TB2_XBfq.dnpuFjSZPhXXbChpXa_!!920073637.jpg as 18.jpg.
Saved image

Saved image: 54f7c594Nc4e47327.jpg as 131.jpg.
Saved image: 5874a1e9N9045b515.jpg as 132.jpg.
Saved image: TB2axZSpl0kpuFjy1zdXXXuUVXa_!!1981917970.jpg as 133.jpg.
Saved image: TB2qtbidOKO.eBjSZPhXXXqcpXa_!!1953837607.png as 134.jpg.
Saved image: rBEhV1KbTWYIAAAAAAD0V-1QmtkAAGUMgIeVAgAAPRv477.jpg as 135.jpg.
Saved image: CgQCtVMYCD-AO8idAAGpfajCQfA47800_380x380.jpg as 136.jpg.
Saved image: TB23geIb0HO8KJjSZFHXXbWJFXa_!!1087100576.jpg as 137.jpg.
Saved image: 567228beN6c2a61bd.jpg as 138.jpg.
Saved image: 687474703a2f2f6777312e616c6963646e2e636f6d2f62616f2f75706c6f616465642f69352f5431772e486e466f6c6558585858585858585f2121302d6974656d5f7069632e6a7067.jpg as 139.jpg.
Saved image: 5785b667N3ae1818e.jpg as 140.jpg.
Saved image: 687474703a2f2f6777312e616c6963646e2e636f6d2f62616f2f75706c6f616465642f69352f544231715a387949465858585863435846585858585858585858585f2121302d6974656d5f7069632e6a7067.jpg as 141.jpg.
Saved image: 14690503191205.jpg as 142.jpg.
Saved image: T1D8mPFddbXXXXXXXX_!!0-item_p

Saved image: 5869ec59N74c01a3f.jpg as 256.jpg.
Saved image: 0070161976-000000000621802614_4_800x800.jpg as 257.jpg.
Saved image: 5a616ba2N2cd93e0f.jpg as 258.jpg.
Saved image: TB20k7saqmgSKJjSsphXXcy1VXa_!!1037465325.jpg as 259.jpg.
Saved image: TB2GZh.mb8kpuFjy0FcXXaUhpXa_!!1029154337.jpg as 260.jpg.
Saved image: 56d6957aN6ccad361.jpg as 261.jpg.
Saved image: b0af91f821d79bf984a34fbcb11b07a7.jpg as 262.jpg.
Saved image: 5721a236N3bc45ac9.jpg as 263.jpg.
Saved image: TB15EsSFFXXXXcTXVXXXXXXXXXX_%21%210-item_pic.jpg as 264.jpg.
Saved image: 201311300325148334.gif as 265.jpg.
Saved image: TB289t8fpXXXXXXXpXXXXXXXXXX_!!2608936771.jpg as 266.jpg.
Saved image: 53741100229_634.jpg as 267.jpg.
Saved image: 59ab96b6N6543cbaa.jpg as 268.jpg.
Saved image: TB1eul8IVXXXXbgXXXXXXXXXXXX_!!0-item_pic.jpg_460x460xz.jpg as 269.jpg.
Saved image: CgQIzlUKnyuAITIKAANJSbA9btE37800_600x600.jpg as 270.jpg.
Saved image: 59019927N561c22c2.jpg as 271.jpg.
Saved image: CgQDrVHU4uKAHm7qAAF497gGiQ079400.jpg as 272

Saved image: 55d57968N140c21f6.jpg%21q70.jpg as 383.jpg.
Saved image: 55bc6a37Nb34552c6.jpg as 384.jpg.
Saved image: TB2SRlntmtkpuFjy0FhXXXQzFXa_!!2913430649.jpg as 385.jpg.
Saved image: 57df5794N62c246b4.jpg as 386.jpg.
Saved image: CgMBmVNqb9OAMNt7AADXUZwsDaE75600.jpg as 387.jpg.
Saved image: 5a13cee3Nd91c0cb0.jpg as 388.jpg.
Saved image: 59784242Na22770c2.jpg as 389.jpg.
Saved image: 59608db8N5968cb7a.jpg as 390.jpg.
Saved image: 0070063805-000000000104790241_3_800x800.jpg as 391.jpg.
Saved image: TB2ZK3_tR0kpuFjSsziXXa.oVXa_!!1021578509.png as 392.jpg.
Saved image: 5831b767Ne178f150.jpg as 393.jpg.
Saved image: 2102199810.jpg as 394.jpg.
Saved image: 5976fdcdN299ec739.jpg as 395.jpg.
Saved image: TB2BbpmhhXkpuFjy0FiXXbUfFXa_!!268691557.jpg as 396.jpg.
Saved image: TB1kAT5HFXXXXamXVXXXXXXXXXX_!!0-item_pic.jpg as 397.jpg.
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Connection refused by the server..
Let me sleep for

Saved image: 5763df7dN63db8b8d.jpg%21q70.jpg as 508.jpg.
Saved image: 58039c7fN8a8ee782.jpg as 509.jpg.
Saved image: 59f5a7a5N5afd968c.jpg as 510.jpg.
Saved image: TB2P7urbvL85uJjSZFyXXa93XXa_!!3039044872.jpg as 511.jpg.
Saved image: TB1FyvQLVXXXXbcaXXXXXXXXXXX_!!0-item_pic.jpg as 512.jpg.
Saved image: 201411184489763.jpeg as 513.jpg.
Saved image: CgQCrlOZXUyAdN2UAAKqapLBky032000_450x450.jpg as 514.jpg.
Saved image: 614bBnDHW8L._SL1000_.jpg as 515.jpg.
Saved image: a1ec08fa513d2697d491a7675efbb2fb4316d84e.jpg as 516.jpg.
Saved image: 1317554806-2_w_1.jpg as 517.jpg.
Saved image: 5823ee6bN701a0768.jpg as 518.jpg.
Saved image: CgQIz1QBkwSAW7NRAAQ-aXNlNrk93700.jpg as 519.jpg.
Saved image: 59190288N75ad2164.jpg as 520.jpg.
Saved image: 55df579cN4f02d4a1.jpg as 521.jpg.
Saved image: 56cd5257Ne2114c9e.jpg as 522.jpg.
Saved image: 540133d3Nde579995.jpg%21q70.jpg as 523.jpg.
Saved image: 000000000104755662_2f.jpg as 524.jpg.
Saved image: 0070143179-000000000605443846_3_800x800.jpg as 525.jpg.


...

Saved image: TB2aMr3dHsTMeJjSszhXXcGCFXa_!!2278404391.jpg as 24370.jpg.
Saved image: 599dc1d2N2b5e2bb1.jpg as 24371.jpg.
Saved image: 553a2f68N1dc13cd9.jpg%21q70.jpg as 24372.jpg.
Saved image: 687474703a2f2f6777322e616c6963646e2e636f6d2f62616f2f75706c6f616465642f69322f5442313776323351705858585858675856585858585858585858585f2121302d6974656d5f7069632e6a7067.jpg as 24373.jpg.
Saved image: 5938adc2N250bb848.jpg as 24374.jpg.
Saved image: 57be8c43Na1edafc0.jpg as 24375.jpg.
Saved image: TB2l0xKpVXXXXXpXXXXXXXXXXXX_%21%2155372027.gif as 24376.jpg.
Saved image: 1-16122Q4150U02.jpg as 24377.jpg.
Saved image: 224ab9067d6152f1870914667bf3cfec.png as 24378.jpg.
Saved image: 58acef8cNd7446e93.jpg as 24379.jpg.
Saved image: T1.F7YXX8eXXXXXXXX_%21%210-item_pic.jpg_310x310.jpg as 24380.jpg.
Saved image: 57721235Nda453a44.jpg as 24381.jpg.
Saved image: TB2Cqvih_vI8KJjSspjXXcgjXXa_!!1052284186.jpg as 24382.jpg.
Saved image: 5604b0d7Nc0b55f84.jpg%21q70.jpg as 24383.jpg.
Saved image: 59c6441fNf1b295d8.jp

Saved image: 5905943aNe6f35b04.jpg as 24484.jpg.
Saved image: 20160318090644062.jpg as 24485.jpg.
Saved image: 5a45abe0N40e100ae.jpg as 24486.jpg.
Saved image: 1-1F6031J60aS.jpg as 24487.jpg.
Saved image: 20131031235312-5ffe6695.jpg as 24488.jpg.
Saved image: 5553000cN7bd42811.jpg%21q70.jpg as 24489.jpg.
Saved image: 5a28f7e9N9c7fd900.jpg as 24490.jpg.
Saved image: 56fcdccbN3049756a.jpg as 24491.jpg.
Saved image: 586660d9N47c237ca.jpg as 24492.jpg.
Saved image: 57471af0N92a5a236.jpg as 24493.jpg.
Saved image: rBEGF1CROQQIAAAAAAHiWTM-wmYAAASwACPpoIAAeJx272.jpg as 24494.jpg.
Saved image: TB2RQC.fPnD8KJjSspbXXbbEXXa_!!1029861720.jpg as 24495.jpg.
Saved image: 574d3105Ndc827556.jpg as 24496.jpg.
Saved image: 5950da09N28a9914b.jpg as 24497.jpg.
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Saved image: f4aaaac6-daac-4717-a65e-9a9a42a97978.jpg as 24498.jpg.
Saved image: 58f9a3c3N9167670c.jpg as 24499.jpg.
Saved image: 58f8698

Saved image: MTQ2OTczMTQwMzY2MTQzNTUyMzAyOA==.jpg as 24613.jpg.
Saved image: TB2gNJbgrBnpuFjSZFGXXX51pXa_!!2536915372.jpg as 24614.jpg.
Saved image: 104105_74106.jpg as 24615.jpg.
Saved image: TB2yg96bVXXXXbSXpXXXXXXXXXX-731374351.jpg as 24616.jpg.
Saved image: 59361e6bN58580758.jpg as 24617.jpg.
Saved image: 09241313694_800x800.jpg_375_375.jpg as 24618.jpg.
Saved image: 58d63fa5N94eacc91.jpg as 24619.jpg.
Saved image: CgQIzVfZj9-ANSVbAAOUtZGQBug00600_320x320.jpg as 24620.jpg.
Saved image: TB1EBnaJFXXXXcRXpXXXXXXXXXX_!!0-item_pic.jpg_460x460xz.jpg as 24621.jpg.
Saved image: TB2IPSGtFXXXXcYXpXXXXXXXXXX_!!2807020406.gif as 24622.jpg.
Saved image: TB2iKh8dmiJ.eBjSspfXXbBKFXa_!!2808049988.jpg as 24623.jpg.
Saved image: tujiJDAzJDI0L1RCMWlSVC5LJDZYWFlhJDYkNlhYWF8hISQz.jpg as 24624.jpg.
Saved image: TB2VcbIaL6H8KJjSspmXXb2WXXa_!!656825722.jpg as 24625.jpg.
Saved image: 6361058964772623844896449.jpg as 24626.jpg.
Saved image: TB2ab5jivDH8KJjy1XcXXcpdXXa_!!2868167413.jpg as 24627.jpg.
Saved im

Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Failed to get 24729.jpg
Saved image: TB270Tpczgy_uJjSZKbXXXXkXXa_!!44634772.jpg as 24730.jpg.
Saved image: TB2ULgOsmFmpuFjSZFrXXayOXXa_!!2611161899.jpg as 24731.jpg.
Saved image: 5777253fNf40fbfab.jpg as 24732.jpg.
Saved image: 71954995_1515161155744.jpg as 24733.jpg.
Saved image: 5a157e01N362ae982.jpg as 24734.jpg.
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Connection refused by the server..
Let me sleep for 5 seconds
ZZzzzz...
Was a nice sleep, now let me continue...
Fail

Saved image: TB2iRMbXrYI8KJjy0FaXXbAiVXa_!!1749377410.jpg as 24840.jpg.
Saved image: 59ae6450Nbd3c7083.jpg as 24841.jpg.
Saved image: 53c48d7aN25586f2f.jpg as 24842.jpg.
Saved image: small_201412231329063129.jpg as 24843.jpg.
Saved image: TB2LuImX9Ib61BjSZFDXXa0yVXa_!!67408832.jpg as 24844.jpg.
Saved image: wKhQdFSXoZOEA_TRAAAAAGK3ZJg495.jpg as 24845.jpg.
Saved image: 2e32f0b3-e4c8-41a1-b93b-2b48c460b833.jpg as 24846.jpg.
Saved image: 5800854dNf5960b5a.jpg as 24847.jpg.
Saved image: T1K4BIXd4OXXav.tZ7_064248.jpg as 24848.jpg.
Saved image: 1173828412-1_w_1.jpg as 24849.jpg.
Saved image: 58bba7dbN0eee430e.jpg as 24850.jpg.
Saved image: 5a138896Ndb9cc725.jpg as 24851.jpg.
Saved image: 58ff02ffN76b90bc1.jpg as 24852.jpg.
Saved image: 1n0i4a_ifrtgzlemi3taobrgyzdambqhayde_640x498.jpg as 24853.jpg.
Saved image: TB1_I26QVXXXXaPXpXXXXXXXXXX_!!0-item_pic.jpg_310x310.jpg as 24854.jpg.
Saved image: 55ab6fcfN15baefac.jpg as 24855.jpg.
Saved image: TB2fQ2vaXXXXXb_XXXXXXXXXXXX_%21%21667801421-0-MATRI

In [ ]:
image_url_df = parse_json(valid_image_file, 'images')
image_annotations_df = parse_json(valid_image_file, 'annotations')
image_annotations_df.to_csv(JSON_FILE_PATH + "train_label.csv",
                            index=False)

In [ ]:
save_image_batch(image_url_df,"validation",1)

In [ ]:
image_url_df = parse_json(test_image_file, 'images')

save_image_batch(image_url_df,"test",1)